🟩 1. Translation (Tịnh tiến)


In [ ]:
import cv2
import numpy as np

img = cv2.imread('input.jpg')
rows, cols = img.shape[:2]

# Ma trận tịnh tiến: sang phải 50, xuống 30
M = np.float32([[1, 0, 50], [0, 1, 30]])
translated = cv2.warpAffine(img, M, (cols, rows))

cv2.imwrite('translated.jpg', translated)


🟦 2. Euclidean (Rigid Transform: Xoay + tịnh tiến)


In [ ]:
# Xoay 45 độ quanh tâm ảnh, scale = 1
center = (cols // 2, rows // 2)
M = cv2.getRotationMatrix2D(center, angle=45, scale=1.0)
rotated = cv2.warpAffine(img, M, (cols, rows))

cv2.imwrite('rigid_transform.jpg', rotated)


🟨 3. Similarity (Xoay + scale + tịnh tiến)


In [ ]:
# 3 điểm trước và sau để tính similarity
pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
pts2 = np.float32([[70, 70], [230, 60], [70, 230]])

M = cv2.estimateAffinePartial2D(pts1, pts2)[0]
similarity = cv2.warpAffine(img, M, (cols, rows))

cv2.imwrite('similarity.jpg', similarity)


🟧 4. Affine (Biến đổi tuyến tính tổng quát)


In [ ]:
# 3 điểm bất kỳ
pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
pts2 = np.float32([[10, 100], [200, 50], [100, 250]])

M = cv2.getAffineTransform(pts1, pts2)
affine = cv2.warpAffine(img, M, (cols, rows))

cv2.imwrite('affine.jpg', affine)


🟥 5. Projective (Phép biến đổi phối cảnh – Perspective)
python
Sao chép
Chỉnh sửa


In [ ]:
# 4 điểm tương ứng cho perspective transform
pts1 = np.float32([[0, 0], [300, 0], [0, 300], [300, 300]])
pts2 = np.float32([[10, 50], [280, 10], [50, 290], [290, 290]])

M = cv2.getPerspectiveTransform(pts1, pts2)
perspective = cv2.warpPerspective(img, M, (cols, rows))

cv2.imwrite('projective.jpg', perspective)


❌ 1. Forward Warping (Tự viết – dễ gây lỗi “holes”)


In [ ]:
import cv2
import numpy as np


def forward_warping(img, M, output_shape):
    h, w = img.shape[:2]
    output = np.zeros((output_shape[1], output_shape[0], 3), dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            src_pos = np.array([x, y, 1])
            dst_pos = M @ src_pos
            dst_x, dst_y = int(dst_pos[0]), int(dst_pos[1])
            if 0 <= dst_x < output_shape[0] and 0 <= dst_y < output_shape[1]:
                output[dst_y, dst_x] = img[y, x]

    return output


# Ví dụ sử dụng
img = cv2.imread('input.jpg')
rows, cols = img.shape[:2]
# Ma trận dịch phải 50px, xuống 30px
M = np.array([[1, 0, 50], [0, 1, 30], [0, 0, 1]])

warped = forward_warping(img, M, (cols + 100, rows + 100))
cv2.imwrite('forward_warping.jpg', warped)


✅ 2. Inverse Warping (Chuẩn bài – dùng nội suy)


In [ ]:
def inverse_warping(img, M_inv, output_shape):
    h_out, w_out = output_shape
    output = np.zeros((h_out, w_out, 3), dtype=np.uint8)

    for y in range(h_out):
        for x in range(w_out):
            dst_pos = np.array([x, y, 1])
            src_pos = M_inv @ dst_pos
            src_x, src_y = src_pos[0], src_pos[1]

            if 0 <= src_x < img.shape[1]-1 and 0 <= src_y < img.shape[0]-1:
                # Bilinear interpolation
                x0, y0 = int(src_x), int(src_y)
                dx, dy = src_x - x0, src_y - y0
                val = (1 - dx) * (1 - dy) * img[y0, x0] + \
                    dx * (1 - dy) * img[y0, x0+1] + \
                    (1 - dx) * dy * img[y0+1, x0] + \
                    dx * dy * img[y0+1, x0+1]
                output[y, x] = np.clip(val, 0, 255)

    return output


# Sử dụng:
img = cv2.imread('input.jpg')
rows, cols = img.shape[:2]
# Ma trận dịch: phải 50, xuống 30
M = np.array([[1, 0, 50], [0, 1, 30], [0, 0, 1]])
M_inv = np.linalg.inv(M)

warped_inv = inverse_warping(img, M_inv, (rows + 100, cols + 100))
cv2.imwrite('inverse_warping.jpg', warped_inv)
